#### Datansatz und Datenimport

In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [6]:
df.reset_index(inplace= True)

In [7]:
df.head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


### **P1:** Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120

In [ ]:
df.loc[df.product_id.isin([24852, 24964, 2120])]

: 

: 

In [ ]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True)

: 

: 

In [ ]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True, normalize= "index")

: 

: 

In [ ]:
df_tip1 = df.loc[df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip1.head()

: 

: 

In [ ]:
df_tip1.loc[df_tip1.tip == 1]["tip"].count() / df_tip1.tip.count()

: 

: 

In [ ]:
df_tip1.loc[df_tip1.tip == 0]["tip"].count() / df_tip1.tip.count()

: 

: 

In [ ]:
df_tip2 = df.loc[~df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip2.head()

: 

: 

In [ ]:
df_tip2.loc[df_tip2.tip == 1]["tip"].count() / df_tip2.tip.count()

: 

: 

In [ ]:
df_tip2.loc[df_tip2.tip == 0]["tip"].count() / df_tip2.tip.count()

: 

: 

In [ ]:
df_tip1.corr()

: 

: 

In [ ]:
df_tip2.corr()

: 

: 

In [ ]:
# Chi Quadrat Test

: 

: 

### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [ ]:
df_county_tip = df.groupby("county").agg(tip = ("tip", "sum"), tip_opp = ("tip", "count")).reset_index()
df_county_tip["no_tip"] = df_county_tip.tip_opp - df_county_tip.tip

: 

: 

In [ ]:
df_county_tip["tip_norm"], df_county_tip["no_tip_norm"] = [df_county_tip.iloc[:, 1:2].apply(lambda x: x / df_county_tip.tip_opp, axis= 0), df_county_tip.iloc[:, 3:4].apply(lambda x: x / df_county_tip.tip_opp, axis= 0)]

: 

: 

In [ ]:
df_county_tip.drop(["tip_opp"], axis= 1, inplace= True)

: 

: 

In [ ]:
df_county_tip.head()

: 

: 

In [ ]:
df_county_tip.loc[df_county_tip.tip_norm > df_county_tip.no_tip_norm]

: 

: 

In [ ]:
pd.crosstab(df.county, df.tip, margins= True, normalize= False).head()

: 

: 

In [ ]:
df_county_tip_sorted = df_county_tip.sort_values(by= ["tip_norm"], ascending= False)
px.bar(df_county_tip_sorted, x= "county", y= "tip_norm")

: 

: 

### **P3:** Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

#### Correlation

In [8]:
df.columns

Index(['index', 'order_id', 'product_id', 'add_to_cart_order', 'reordered',
       'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'tip', 'product_name', 'aisle_id',
       'department_id', 'department', 'aisle', 'county'],
      dtype='object')

In [78]:
df[["product_id", "reordered", "order_dow", "order_hour_of_day", "days_since_prior_order", "tip"]].corr()

,product_id,reordered,order_dow,order_hour_of_day,days_since_prior_order,tip
product_id,1.000000,0.003051,-0.002709,0.001654,0.000652,-0.001380
reordered,0.003051,1.000000,-0.011324,-0.022744,-0.130482,0.077467
order_dow,-0.002709,-0.011324,1.000000,0.009725,-0.030760,-0.114064
order_hour_of_day,0.001654,-0.022744,0.009725,1.000000,0.004329,0.025336
days_since_prior_order,0.000652,-0.130482,-0.030760,0.004329,1.000000,-0.143219
tip,-0.001380,0.077467,-0.114064,0.025336,-0.143219,1.000000


In [10]:
df.loc[df.tip == 1].head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


#### Order_dow

In [172]:
df1 = df.groupby(["order_dow", "order_id"], as_index= False).agg(tip_orders_per_day = ("tip", "mean"))
df1

,order_dow,order_id,tip_orders_per_day
0,0,4,1.0
1,0,6,0.0
2,0,9,1.0
3,0,10,0.0
4,0,19,1.0
...,...,...,...
606910,6,606886,1.0
606911,6,606898,1.0
606912,6,606904,0.0
606913,6,606906,1.0


In [182]:
df_order_dow= df1.groupby("order_dow", as_index= False).agg(tip = ("tip_orders_per_day","sum"), order_id= ("order_id", "nunique"))
df_order_dow["tip_norm"] = df_order_dow.tip / df_order_dow.order_id
df_order_dow

,order_dow,tip,order_id,tip_norm
0,0,53749.0,106842,0.503070
1,1,54928.0,104713,0.524558
2,2,30614.0,83177,0.368058
3,3,27884.0,77329,0.360589
4,4,27780.0,75323,0.368812
5,5,30188.0,80360,0.375660
6,6,28234.0,79171,0.356620


**Tippwahrscheinlichkeit für jeden Wochentag**

In [185]:
px.bar(df_order_dow, x= "order_dow", y= "tip_norm")

#### Order_hour_of_day

In [201]:
df_h = df.groupby(["order_hour_of_day", "order_id"], as_index= False).agg(tip= ("tip", "mean"))
df_hour = df_h.groupby(["order_hour_of_day"], as_index= False).agg(order_id= ("order_id", "nunique"), tip= ("tip", "sum"))
df_hour["tip_norm"] = df_hour.tip / df_hour.order_id
df_hour

,order_hour_of_day,order_id,tip,tip_norm
0,0,4072,2079.0,0.510560
1,1,2219,1125.0,0.506985
2,2,1340,694.0,0.517910
3,3,956,477.0,0.498954
4,4,986,510.0,0.517241
5,5,1644,619.0,0.376521
6,6,5461,2146.0,0.392968
7,7,16293,6641.0,0.407598
8,8,31847,13411.0,0.421107
9,9,45728,19502.0,0.426478


**Tippwahrscheinlichkeit pro Stunde**

In [202]:
px.bar(df_hour, x="order_hour_of_day", y="tip_norm")

#### user_id

In [207]:
df_user = df.groupby(["user_id", "order_id"], as_index= False).agg(user_tip_mean= ("tip", "mean"))
df_user_tip = df_user.groupby("user_id", as_index= False).agg(tip= ("user_tip_mean", "sum"), orders = ("order_id", "nunique"))
df_user_tip["tip_norm"] = df_user_tip.tip / df_user_tip.orders
df_user_tip

,user_id,tip,orders,tip_norm
0,3,10.0,12,0.833333
1,7,12.0,21,0.571429
2,14,8.0,14,0.571429
3,23,0.0,5,0.000000
4,25,2.0,3,0.666667
...,...,...,...,...
37362,206186,2.0,4,0.500000
37363,206189,2.0,6,0.333333
37364,206192,2.0,14,0.142857
37365,206195,3.0,20,0.150000


In [ ]:
# pd.cut machen

#### product_name

In [42]:
df_product_tip = df.groupby("product_name", as_index= False).agg(tip_per_product= ("tip", "sum"), orders= ("index", "count")).sort_values(by= "tip_per_product", ascending= False)

In [45]:
df_product_tip["tip_rate_per_product"] = df_product_tip.tip_per_product / df_product_tip.orders
df_product_tip.loc[df_product_tip.orders >= 10000]

,product_name,tip_per_product,orders,tip_rate_per_product
3369,Banana,48084,89127,0.539500
3177,Bag of Organic Bananas,37468,70283,0.533102
29385,Organic Strawberries,23033,49961,0.461020
26443,Organic Baby Spinach,19480,45462,0.428490
27836,Organic Hass Avocado,18013,39290,0.458463
26408,Organic Avocado,14419,32896,0.438321
20554,Large Lemon,12205,28495,0.428321
29920,Organic Whole Milk,12078,25309,0.477222
28851,Organic Raspberries,11907,25589,0.465317
21486,Limes,11608,26443,0.438982


#### Order_size

In [61]:
df_order_size = df.groupby("order_id", as_index= False).agg(order_size= ("order_id", "count"), tip= ("tip", "sum"))
df_order_size["tip"] = df_order_size.tip / df_order_size.order_size
df_order_size

,order_id,order_size,tip
0,1,8,1.0
1,2,3,0.0
2,3,5,0.0
3,4,3,1.0
4,5,2,0.0
...,...,...,...
606910,606911,6,0.0
606911,606912,13,1.0
606912,606913,19,0.0
606913,606914,8,1.0


In [57]:
df_order_size.loc[df_order_size.tip == 1].order_size.mean()

10.831081747751375

In [58]:
df_order_size.loc[df_order_size.tip == 0].order_size.mean()

9.584039056621918

In [64]:
df_order_ = df.groupby("order_id", as_index= False).agg(order_size= ("order_number", "max"), tip= ("tip", "max"))
df_order_.loc[df_order_.tip == 1].order_size.mean()

18.91474758956022

In [65]:
df_order_.loc[df_order_.tip == 0].order_size.mean()

15.851800937947264

#### days_since_prior_order

In [74]:
df_days_since_prior_order = df.groupby("days_since_prior_order", as_index= False).agg(tip= ("tip", "sum"), all= ("index", "count"))
df_days_since_prior_order["tip_norm"] = df_days_since_prior_order["tip"] / df_days_since_prior_order["all"]
df_days_since_prior_order

,days_since_prior_order,tip,all,tip_norm
0,0.0,37944,84279,0.450219
1,1.0,71558,174617,0.409800
2,2.0,118475,271301,0.436692
3,3.0,160890,342541,0.469696
4,4.0,184265,381056,0.483564
5,5.0,196368,393925,0.498491
6,6.0,244190,465038,0.525097
7,7.0,378065,651494,0.580305
8,8.0,189581,362074,0.523597
9,9.0,108699,228319,0.476084


In [77]:
px.bar(df_days_since_prior_order.sort_values(by= "tip_norm", ascending= False), x= "days_since_prior_order", y= "tip_norm")

#### department

In [161]:
df_dep = df.groupby(["department_id", "order_id"], as_index= False).agg(tip= ("tip", "mean"), all= ("order_id", "count"))
df_dep

,department_id,order_id,tip,all
0,1,4,1.0,3
1,1,5,0.0,1
2,1,9,1.0,2
3,1,10,0.0,1
4,1,11,0.0,1
...,...,...,...,...
2883689,21,606641,1.0,1
2883690,21,606646,1.0,3
2883691,21,606806,0.0,1
2883692,21,606807,1.0,1


In [162]:
df_dep.groupby("department_id").agg(sum = ("tip", "sum"))

,sum
department_id,
1,82353.0
2,2914.0
3,69343.0
4,188955.0
5,12531.0
6,19089.0
7,116397.0
8,4310.0
9,43500.0


In [164]:
df_dep = df.groupby("department_id", as_index= False).agg(tip= ("tip", "sum"), all= ("tip", "count"))
df_dep["tip_norm"] = df_dep["tip"] / df_dep["all"]
df_dep

,department_id,tip,all,tip_norm
0,1,158199,428037,0.369592
1,2,3019,6982,0.432398
2,3,94193,223643,0.421176
3,4,784791,1773903,0.442409
4,5,23198,28347,0.818358
5,6,23094,51067,0.452229
6,7,221391,511550,0.432785
7,8,6941,18563,0.373916
8,9,63375,163807,0.386888
9,10,3541,6777,0.522503


In [165]:
df.groupby("department_id", as_index= False).agg(tip= ("tip", "mean"))

,department_id,tip
0,1,0.369592
1,2,0.432398
2,3,0.421176
3,4,0.442409
4,5,0.818358
5,6,0.452229
6,7,0.432785
7,8,0.373916
8,9,0.386888
9,10,0.522503


### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

In [111]:
df_userh = df.groupby("user_id", as_index= False).agg(orders = ("order_id", "nunique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))

In [128]:
df_userh.loc[df_userh.orders >= df_userh.orders.quantile(0.8)]

,user_id,orders,reordered,tip
27,140,77,0.739353,0.527257
40,195,59,0.801047,0.769634
116,690,100,0.764228,0.966318
143,854,84,0.730469,0.135417
150,898,53,0.596226,0.166038
...,...,...,...,...
37191,205237,99,0.775455,0.574511
37244,205496,87,0.810726,0.742376
37249,205513,73,0.757116,0.275142
37316,205917,62,0.705463,0.000000


In [135]:
list_bins = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df_userh["orders_cut"] = pd.cut(df_userh["orders"], bins= list_bins, include_lowest= True)

In [148]:
df_userh1 = df_userh.groupby("orders_cut", as_index= False).agg(orders= ("orders", "sum"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))
df_userh1

,orders_cut,orders,reordered,tip
0,"(0.999, 10.0]",118378,0.323849,0.269258
1,"(10.0, 20.0]",131584,0.499379,0.357115
2,"(20.0, 30.0]",95863,0.598196,0.453186
3,"(30.0, 40.0]",70337,0.653196,0.517118
4,"(40.0, 50.0]",56223,0.690863,0.548516
5,"(50.0, 60.0]",40602,0.718624,0.539258
6,"(60.0, 70.0]",28111,0.713687,0.512011
7,"(70.0, 80.0]",18689,0.733060,0.467081
8,"(80.0, 90.0]",12513,0.764731,0.496327
9,"(90.0, 100.0]",34615,0.759736,0.409032


In [158]:
df_userh1["orders_cut"] = df_userh1["orders_cut"].astype("str")

In [160]:
px.bar(df_userh1, x= "orders_cut", y= "tip", color= "reordered")